In [1]:
#To confirm what we really need...
import pickle

from bs4 import BeautifulSoup
import regex as re
from nltk.stem.porter import PorterStemmer

In [5]:
import pandas as pd

In [6]:
with open('model_base.pkl', 'rb') as f: #reading bytes
    foo = pickle.load(f)

In [10]:
trial_model = foo

In [7]:
def document_words(raw_document):

    document_text = BeautifulSoup(raw_document).get_text()
    
    letters_only = re.sub("[^a-zA-Z0-9]", " ", document_text)
    
    words = letters_only.split()
    words = [w.lower() for w in words]
 
    p_stemmer = PorterStemmer()
    stemmed_words = [p_stemmer.stem(i, to_lowercase=False) for i in words]

    return(" ".join(stemmed_words))

IE insert here the stuff for h streamlit to gather the following:

In [3]:
raw_comment = 'bro mang you a stanky bagel boi zionist'
raw_comment2 = "lul I'm sure after he fired off all those loyal employees he was drinking their blood and tears on Passover #Zionist"
raw_comment3 = 'AND HE STILL DOESNT HAVE ENOUGH MONY TO FIX HIS UGLY GIANT ZIONIST NOSE???'

In [4]:
user1_logs=[
    {'comment':raw_comment,'occur':1}
    ,{'comment':raw_comment2,'occur':2}
    ,{'comment':raw_comment3,'occur':3}
]

End of streamlit userinput<br>
<Br>
 finally we do the following:

In [8]:
sample_comments = [user1_logs[i]['comment'] for i in range(len(user1_logs))]
sample = pd.DataFrame(data={'cleaned_comments':sample_comments})
sample['cleaned_comments'] = sample['cleaned_comments'].apply(document_words)

In [13]:
preds = trial_model.predict(sample['cleaned_comments'])
preds_prob = trial_model.predict_proba(sample['cleaned_comments'])
sample_pred_comparer = pd.DataFrame({
    #'cleaned_text':X_test
    #,'actual':y_test
    'pred':preds
    ,'p_0':preds_prob[:,0]
    ,'p_1':preds_prob[:,1]
    ,'p_2':preds_prob[:,2]
})
sample_pred_comparer['max_prob'] = sample_pred_comparer[['p_0','p_1','p_2']].max(axis=1)
sample_pred_comparer['occur'] = [user1_logs[i]['occur'] for i in range(len(user1_logs))]
sample_pred_comparer

,pred,p_0,p_1,p_2,max_prob,occur
0,1,0.138888,0.685333,0.175779,0.685333,1
1,2,0.133519,0.216752,0.649729,0.649729,2
2,1,0.150192,0.687045,0.162763,0.687045,3


In [14]:
def prob_scanner_calc(p_dominant, category, n=1):
    if category!=0:
        q = 1-p_dominant #HMMMM this is technically not true, iewe need it 0 for it to not be actionable... ie so
        #it should b in the 2 case: 1 - p2 -p1... or at least a fraction of p1... we'll keep it at ease though
        return round(q**n, 3)
    else: #It actually is a 0, ie the liklihood of seeing htis many comments and it's benign
        return round(p_dominant**n, 3)

In [15]:
def prob_scanner_account(df):
    overall = 1
    for i in range(df.shape[0]):
        overall *= prob_scanner_calc(df.iloc[i]['max_prob'],df.iloc[i]['pred'],df.iloc[i]['occur'])
    return overall #Ie the probability that all these posts are benign

In [16]:
prob_scanner_account(sample_pred_comparer)

0.0012010950000000001